In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName("EnergyConsumptionAnalysis").getOrCreate()

In [ ]:
import boto3
import requests

In [ ]:
data_url = "https://data.open-power-system-data.org/time_series/opsd-time_series-2020-10-06.zip"

s3_bucket = "aws-emr-studio-624855134767-us-east-1"
s3_key = "data/opsd-time_series.zip"  

In [ ]:
s3_client = boto3.client("s3")
response = requests.get(data_url, stream=True)

s3_client.upload_fileobj(response.raw, s3_bucket, s3_key)


In [ ]:
import zipfile
import os
import io

In [ ]:
s3_object = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
zip_content = io.BytesIO(s3_object["Body"].read())

with zipfile.ZipFile(zip_content, "r") as zip_ref:
    extracted_files = zip_ref.namelist()

    for file_name in extracted_files:
        with zip_ref.open(file_name) as file_data:
            s3_client.upload_fileobj(file_data, s3_bucket, f"data/{file_name}")



In [ ]:
s3_bucket = "aws-emr-studio-624855134767-us-east-1"
s3_prefix = "data/"
response = s3_client.list_objects_v2(Bucket=s3_bucket, Prefix=s3_prefix)

if "Contents" in response:
    for obj in response["Contents"]:
        print(obj["Key"])
else:
    print("No files in the directory")